In [0]:
#Library 
import numpy as np
import pandas as pd

In [0]:
#Reading Dataframe
df = pd.read_csv('data1_19.csv')

In [0]:
#Check if data is of only one class
def IS_Data_Pure(data):
  
  d = data[:,-1]
  arr = np.unique(d)
  if len(arr) == 1:
    return True
  else: return False

In [0]:
#Give the most frequent class
def Classify_it(data):
  uniq_arr, uniq_arr_counts = np.unique(data, return_counts = True,)
  index = np.argmax(uniq_arr_counts)
  return uniq_arr[index]

In [0]:
def entropy(data):
  classes = np.unique(data)
  n_sample = len(data)
  entropy = 0
  for each_class in classes:
    n_class = len(data[data == each_class])
    prob = n_class / n_sample
    #applying formula
    entropy += prob * np.log2(prob)
  
  return -entropy

In [0]:
#Find best feature according to information gain
def Find_Best_Split(df):
  data = df.values
  #initiating the information gain
  best_Information_gain = -99999
  bestt_feature = ''
  
  #for each feature
  for column in df.drop(df.columns[-1],axis = 1).columns:
    
    #entropy of feature
    entropy_feature = entropy(df.values[:,-1])
    Information_gain_first_term = entropy_feature
    Information_gain_second_term = 0

    n_sample = df[column].count()
    
    for value in df[column].unique():
      df_value = df[df[column] == value]
      #entropy of each value
      entropy_value = entropy(df_value.values[:,-1])
      len_df_value = len(df_value)
      coefficient_for_multiplication = len_df_value / n_sample
      # summation of second term of Information gain
      Information_gain_second_term += coefficient_for_multiplication * entropy_value
    
    #Information Gain
    Information_gain = Information_gain_first_term - Information_gain_second_term

    #Find the best_inforamtion_gain
    if Information_gain > best_Information_gain:
      best_Information_gain = Information_gain
      bestt_feature = column
  
  return bestt_feature
      

In [0]:
#Check if the all element are same
def all_element_same(arr,value):
  
  for each in arr:
    if each != value:
      return False
  
  return True

In [0]:
#Predict the class
def Predict(tree,row):
  if not isinstance(tree,dict):
    return tree
  for question in tree.keys():
    words = question.split()
    #node
    node = words[0]
    #value of node
    value = words[2]
    if row[node] == value:
      return Predict(tree[question],row)

In [0]:
def Decision_Tree_Algo(df):

  sub_tree = {}
  
  # Terminating Condition
  if IS_Data_Pure(df.values):
    return Classify_it(df.values)
  
  else:
    #if more than 1 feature are available
    if len(df.columns) > 1:
      #find =ing the feature
      best_feature = Find_Best_Split(df)
      
      for value in df[best_feature].unique():
   
        question = '{} = {}'.format(best_feature,value)
        yes_value_df = df[df[best_feature] == value]
      
        if len(yes_value_df) > 1 and len(yes_value_df.columns) >= 2:
          yes_value_df = yes_value_df.drop(best_feature, axis = 1)
   
          if len(yes_value_df.columns) >=1:
            sub_tree[question] = Decision_Tree_Algo( yes_value_df )
      
      for each in np.unique(df.values[:,-1]):
        if all_element_same(sub_tree.values(),each):
          return each
     
      return sub_tree
    
    else:
      return Classify_it(df.values)

    

In [0]:
#printing the tree
def print_tree(tree,counter=0):
  #checking if we get class or not and printing if it
  if not isinstance(tree, dict):
    print('{} : {}'.format('\t'*counter,tree))
    return
  #checking each instances
  for question in tree.keys():
    print('\n{}{}'.format('\t'*counter,question))
    print_tree(tree[question], counter + 1)
  
  return
    

In [0]:
#Applying the algo
tree = Decision_Tree_Algo(df)
#printing the tree
print_tree(tree)


gender = male

	pclass = 1st

		age = adult
			 : no

		age = child
			 : yes

	pclass = 2nd

		age = adult
			 : no

		age = child
			 : yes

	pclass = 3rd
		 : no

	pclass = crew
		 : no

gender = female

	pclass = 1st
		 : yes

	pclass = 2nd
		 : yes

	pclass = 3rd
		 : no

	pclass = crew
		 : yes


In [0]:
tree

{'gender = female': {'pclass = 1st': 'yes',
  'pclass = 2nd': 'yes',
  'pclass = 3rd': 'no',
  'pclass = crew': 'yes'},
 'gender = male': {'pclass = 1st': {'age = adult': 'no', 'age = child': 'yes'},
  'pclass = 2nd': {'age = adult': 'no', 'age = child': 'yes'},
  'pclass = 3rd': 'no',
  'pclass = crew': 'no'}}

In [0]:
#predicing the tree
Predict(tree,df.loc[200].drop('survived'))

'yes'

In [0]:
df.loc[200]

pclass         1st
age          adult
gender      female
survived       yes
Name: 200, dtype: object